In [13]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Gatech/Current Classes/CS7643/FinalProject

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Gatech/Current Classes/CS7643/FinalProject


In [14]:
%cd data

/content/drive/My Drive/Gatech/Current Classes/CS7643/FinalProject/data


In [15]:
%ls

preprocessed_data/  test/  train/


In [16]:
import xml.etree.ElementTree as ET
import os
import shutil
import tdqm

In [17]:
file_path = 'train/INKML/CROHME2019/001-equation000.inkml'

namespace = {'ink': 'http://www.w3.org/2003/InkML'}

# Parse the InkML file
tree = ET.parse(file_path)
root = tree.getroot()

truth_values = []

annotation = root.find('.//ink:annotation[@type="truth"]', namespace)

truth_values.append(annotation.text.strip())

In [18]:
print(truth_values)

['$y = Ax + A^2$']


In [19]:
folder_path_IMG = 'train/IMG/CROHME2019'
folder_path_INKML = 'train/INKML/CROHME2019'

files_in_IMG = [os.path.splitext(file)[0] for file in os.listdir(folder_path_IMG)]
files_in_INKML = [os.path.splitext(file)[0] for file in os.listdir(folder_path_INKML)]


folder_path_IMG2 = 'train/IMG/CROHME2023_train'
folder_path_INKML2 = 'train/INKML/CROHME2023_train'

files_in_IMG2 = [os.path.splitext(file)[0] for file in os.listdir(folder_path_IMG2)]
files_in_INKML2 = [os.path.splitext(file)[0] for file in os.listdir(folder_path_INKML2)]

all_files = []

for file1 in files_in_IMG:
    if file1 in files_in_INKML:
        all_files.append(file1)

for file2 in files_in_IMG2:
    if file2 in files_in_INKML2:
        all_files.append(file2)

print(len(all_files))
print(len(files_in_IMG))

12024
10979


In [20]:
files_in_IMG = [os.path.splitext(file)[0] for file in os.listdir(folder_path_IMG)]
files_in_INKML = [os.path.splitext(file)[0] for file in os.listdir(folder_path_INKML)]
print(len(files_in_IMG), len(files_in_INKML))

10979 10984


In [21]:
folder_path = 'train/INKML/CROHME2019'
truth_values = {}
annotations = []

ind = 0
# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.inkml') and filename.split(".")[0] in all_files:
        file_path = os.path.join(folder_path, filename)

        # Parse the InkML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Find and extract truth values from annotations
        namespace = {'ink': 'http://www.w3.org/2003/InkML'}
        annotation = root.find('.//ink:annotation[@type="truth"]', namespace)

        # Extract truth values and append to the list
        annotate = annotation.text.strip()
        truth_values[ind] = {"file": filename.split(".")[0], "annotation": annotate}
        annotations.append(annotate)
        ind += 1

folder_path2 = 'train/INKML/CROHME2023_train'

# Loop through each file in the folder
for filename in os.listdir(folder_path2):
    if filename.endswith('.inkml') and filename.split(".")[0] in all_files:
        file_path = os.path.join(folder_path2, filename)

        # Parse the InkML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Find and extract truth values from annotations
        namespace = {'ink': 'http://www.w3.org/2003/InkML'}
        annotation = root.find('.//ink:annotation[@type="truth"]', namespace)

        # Extract truth values and append to the list
        annotate = annotation.text.strip()
        truth_values[ind] = {"file": filename.split(".")[0], "annotation": annotate}
        annotations.append(annotate)
        ind += 1

print(len(truth_values))

12024


In [22]:
print(truth_values[100]["annotation"])

\sqrt {1 + \sqrt {2 + \sqrt {3 + \sqrt 4}}}


# Now this saves to 2 different folders within train

In [24]:
output_directory = 'preprocessed_data/train/IMG'

output_directory_output = 'preprocessed_data/train/INKML'
img_folder = 'train/IMG/CROHME2019'
img_folder2 = 'train/IMG/CROHME2023_train'

for i in range(len(truth_values)):
    filename = truth_values[i]["file"]
    if i >= 10979:
      img_folder = img_folder2
    image_path = os.path.join(img_folder, filename+".png")

    image_name = f'image_{i}'
    destination_image_path = os.path.join(output_directory, f'{image_name}.png')

    shutil.copyfile(image_path, destination_image_path)

    truth_file_path = os.path.join(output_directory_output, f'truth_value_{i}.txt')
    with open(truth_file_path, 'w') as file:
      file.write(truth_values[i]["annotation"])




# To **HDF5**

In [ ]:
# drive.flush_and_unmount()

In [25]:
!pip install h5py

In [42]:
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tqdm

In [50]:
with h5py.File('train_dataset1.hdf5', 'w') as hdf_file:
    # Create datasets to store images and annotations
    image_group = hdf_file.create_group('images')

    # Iterate through each image path, read the image, and store it in the HDF5 file
    for i in tqdm.tqdm(range(len(all_files))):
        annotation = annotations[i]
        image_path = "preprocessed_data/train/IMG/image_"+str(i)+".png"
        # img = Image.open("preprocessed_data/train/IMG/image_"+str(i)+".png")
        # img_array = np.array(img)
        image_group.create_dataset(f'annotation_{i}', data=np.string_(annotation))
        image_group.create_dataset(f'image_paths_{i}', data=np.string_(image_path))


100%|██████████| 12024/12024 [00:17<00:00, 706.36it/s]


In [52]:
%ls

preprocessed_data/  test/  train/  train_dataset1.hdf5


In [51]:
file_stats = os.stat("train_dataset1.hdf5")
print(file_stats)

os.stat_result(st_mode=33152, st_ino=462973, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=10079475, st_atime=1700367332, st_mtime=1700368600, st_ctime=1700368600)


In [53]:
# Open the HDF5 file in read mode
file_path = 'train_dataset1.hdf5'  # Replace 'your_file.h5' with the path to your HDF5 file
with h5py.File(file_path, 'r') as file:
    # Print the structure of the HDF5 file
    def print_hdf5_structure(item, level=0):
        if isinstance(item, h5py.Group):
            print(f"{'  ' * level}Group: {item.name}")
            for key in item.keys():
                print_hdf5_structure(item[key], level + 1)
        elif isinstance(item, h5py.Dataset):
            print(f"{'  ' * level}Dataset: {item.name} (Shape: {item.shape}, Dtype: {item.dtype})")

    # Print the structure of the file
    print_hdf5_structure(file)

Streaming output truncated to the last 5000 lines.
    Dataset: /images/annotation_4558 (Shape: (), Dtype: |S41)
    Dataset: /images/annotation_4559 (Shape: (), Dtype: |S61)
    Dataset: /images/annotation_456 (Shape: (), Dtype: |S57)
    Dataset: /images/annotation_4560 (Shape: (), Dtype: |S12)
    Dataset: /images/annotation_4561 (Shape: (), Dtype: |S54)
    Dataset: /images/annotation_4562 (Shape: (), Dtype: |S21)
    Dataset: /images/annotation_4563 (Shape: (), Dtype: |S33)
    Dataset: /images/annotation_4564 (Shape: (), Dtype: |S28)
    Dataset: /images/annotation_4565 (Shape: (), Dtype: |S20)
    Dataset: /images/annotation_4566 (Shape: (), Dtype: |S14)
    Dataset: /images/annotation_4567 (Shape: (), Dtype: |S71)
    Dataset: /images/annotation_4568 (Shape: (), Dtype: |S12)
    Dataset: /images/annotation_4569 (Shape: (), Dtype: |S38)
    Dataset: /images/annotation_457 (Shape: (), Dtype: |S8)
    Dataset: /images/annotation_4570 (Shape: (), Dtype: |S55)
    Dataset: /images/a

KeyboardInterrupt: ignored